## Steps

1. **Get markers type from ImageJ coordinates file (`.xml`)**  
   Markers are assigned according to the schema below:
   - **(1)** – pronounced cells in the *expected* area  
   - **(2)** – fuzzy/dim cells in the *expected* area  
   - **(3), (4)** – reference cells  
   - **(5)** – IPR  
   - **(6)** – pronounced cells in the *unexpected* area  

2. **Calibrate points with the reference map**, which includes:
   - Aligning reference point **(3)** with the map equivalent  
   - Scaling according to the **(3), (4)** distance  
   - Rotating according to the **(3), (4)** angle  

3. **3D plot of points**


In [1]:
import xml.etree.ElementTree as ET
import plotly.graph_objects as go
import math
import numpy as np
import os

def get_markers_by_type(root, marker_number):
    """Fuction to retrieve markers by type."""
    x, y, z = [], [], []

    for marker_type in root.findall('.//Marker_Type'):
        type_number = int(marker_type.find('Type').text)
        if type_number == marker_number:
            for marker in marker_type.findall('Marker'):
                x.append(float(marker.find('MarkerX').text))
                y.append(-float(marker.find('MarkerY').text)) # negative Y to match the coordinate system
                z.append(float(marker.find('MarkerZ').text))
    mar_cor = [x, y, z]
    return mar_cor

def calibrate_with_rotation(root, ref_cor):
    """
    Coordinates calibration function with respect to reference points - markers 3 and 4.
    ref_cor: [(x3_ref, y3_ref), (x4_ref, y4_ref), z_ref]
    """
    # Coordinate dictionary for markers 1-6
    cor_dict = {}
    for i in [1,2,3,4,5,6]:
        x,y,z = get_markers_by_type(root, i)
        cor_dict[i] = (x, y, ref_cor[2])

    # Reference point (3) and (4)
    target_x3, target_y3 = ref_cor[0]
    target_x4, target_y4 = ref_cor[1]
    target_y3 = -target_y3  # negative Y to match the coordinate system
    target_y4 = -target_y4  # negative Y to match the coordinate system
    target_z = ref_cor[2] # constant Z for slice

    # Point (3) and (4) in data (imageJ)
    x3, y3, _ = cor_dict[3]
    x4, y4, _ = cor_dict[4]
    x3, y3 = x3[0], y3[0]
    x4, y4 = x4[0], y4[0]

    # --- Scaling
    dx_ref = target_x4 - target_x3
    dy_ref = target_y4 - target_y3
    target_distance = math.sqrt(dx_ref**2 + dy_ref**2) #distance between markers 3 and 4 in reference map

    dx_data = x4 - x3
    dy_data = y4 - y3
    old_distance = math.sqrt(dx_data**2 + dy_data**2) #distance between markers 3 and 4 in data

    scale = target_distance / old_distance

    # --- Rotation
    angle_data = math.atan2(dy_data, dx_data) #angle between markers 3 and 4 in data
    angle_ref = math.atan2(dy_ref, dx_ref) #angle between markers 3 and 4 in reference map
    rotation_angle = angle_ref - angle_data #rotation angle 

    def rotate(x, y, angle_rad): 
        cos_a = np.cos(angle_rad)
        sin_a = np.sin(angle_rad)
        return x * cos_a - y * sin_a, x * sin_a + y * cos_a

    # --- Final calibration
    calibrated_cor_dict = {}
    for key, (x_list, y_list, z_list) in cor_dict.items():
        new_x = []
        new_y = []
        for x, y in zip(x_list, y_list):
            # 1. Align with point 3
            x_shifted = x - x3
            y_shifted = y - y3
            # 2. Skaling
            x_scaled = x_shifted * scale
            y_scaled = y_shifted * scale
            # 3. Rotation
            x_rot, y_rot = rotate(x_scaled, y_scaled, rotation_angle)
            # 4. Final positioning
            x_final = x_rot + target_x3
            y_final = y_rot + target_y3
            new_x.append(x_final)
            new_y.append(y_final)

        # Constant Z
        new_z = [target_z for _ in new_x]
        calibrated_cor_dict[key] = (new_x, new_y, new_z)

    return calibrated_cor_dict

def plot_slice(fig, path, ref_cor, color_map=None, added_types=None):
    """
    Plots a slice from the XML file at the given path, calibrating coordinates with respect to ref_cor.
    """

    if added_types is None:
        added_types = set() #set for legend control

    tree = ET.parse(path)
    root = tree.getroot()
    calibrated_cor_dict = calibrate_with_rotation(root, ref_cor)

    if color_map is None:
        color_map = {1: 'blue', 2: 'lightblue', 5: 'orange', 6: 'purple'}

    for marker_type, (x, y, z) in calibrated_cor_dict.items():
        if marker_type not in color_map:
            continue  # omit markers not in color_map
        color = color_map.get(marker_type, 'black')

        show_legend = False #legend control
        if marker_type not in added_types:
            show_legend = True
            added_types.add(marker_type)

        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=4, color=color, opacity=0.7),
            name=f'Type {marker_type}',
            showlegend=show_legend
        ))

    return added_types

def load_ref_dict_from_txt(folder_path):
    """ 
    Loads a reference dictionary from a text file in the specified folder.
    """

    ref_dict = None
    for file in os.listdir(folder_path):
        if file.endswith('.txt'):
            txt_path = os.path.join(folder_path, file)
            with open(txt_path, 'r') as f:
                text = f.read()
                ref_dict = eval(text)
            break
    return ref_dict

Slice check


In [2]:
number = '5_1'
file_path = f'rat2\\Szkiełko 2\\CellCounter_RMTg_x{number}.xml'

with open('rat2\\Szkiełko 2\\r2s2_cor_ref.txt', 'r') as f:
                text = f.read()
                ref_dict = eval(text)  
ref_cor = ref_dict[number]

tree = ET.parse(file_path)
root = tree.getroot()

calibrated_cor_dict = calibrate_with_rotation(root, ref_cor)
[x,y,z] = calibrated_cor_dict[5]
print([xi for xi in x if xi < 0]) 

color_map = {1: 'blue', 2: 'lightblue', 5: 'orange', 6: 'purple'} 
fig = go.Figure()
plot_slice(fig, file_path, ref_cor, color_map=color_map)

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title=F"RMTG - r1s2, slice{number}",
    legend=dict(x=0.8, y=0.9)
)

fig.show()

[np.float64(-0.33758419243985993), np.float64(-0.278243986254293), np.float64(-0.24393470790377741), np.float64(-0.2166219931271452), np.float64(-0.20485567010309014), np.float64(-0.22623024054982555), np.float64(-0.2082302405498257), np.float64(-0.1823608247422654), np.float64(-0.1812886597938117), np.float64(-0.16363917525772903), np.float64(-0.1482233676975916), np.float64(-0.14579725085910367), np.float64(-0.14099999999999724), np.float64(-0.13970103092783237), np.float64(-0.13961855670102835), np.float64(-0.15139175257731707), np.float64(-0.16205154639175007), np.float64(-0.19029896907216265), np.float64(-0.1997216494845338), np.float64(-0.21154295532645828), np.float64(-0.225804123711338), np.float64(-0.25525429553264395), np.float64(-0.29083505154638967), np.float64(-0.29214089347078814), np.float64(-0.2716666666666648), np.float64(-0.2927800687285207), np.float64(-0.26917869415807383), np.float64(-0.24431958762886413), np.float64(-0.21603092783504957), np.float64(-0.19004467353

All slices - sample 1 i 2

In [3]:
color_map = {1: 'blue', 2: 'lightblue', 5: 'orange'}  # 6: 'purple'
fig = go.Figure()
added_types = set()



#sample 1
folder_path = 'rat2\\Szkiełko 1'
ref_dict = load_ref_dict_from_txt(folder_path)
for filename in os.listdir(folder_path):
    if filename.endswith('.xml'):
        for ref_key in ref_dict.keys():
            if filename.endswith(f"{ref_key}.xml"):
                file_path = os.path.join(folder_path, filename)
                ref_cor = ref_dict[ref_key]       
                added_types = plot_slice(fig, file_path, ref_cor, color_map=color_map, added_types=added_types)

#sample 2
folder_path = 'rat2\\Szkiełko 2'
ref_dict = load_ref_dict_from_txt(folder_path)
for filename in os.listdir(folder_path):
    if filename.endswith('.xml'):
        for ref_key in ref_dict.keys():
            if filename.endswith(f"{ref_key}.xml"):
                file_path = os.path.join(folder_path, filename)
                ref_cor = ref_dict[ref_key]
                added_types = plot_slice(fig, file_path, ref_cor, color_map=color_map, added_types=added_types)

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title="RMTG - rat 2",
    legend=dict(x=0.8, y=0.9)
)

fig.show()

Convex hull - do poprawy

In [4]:
# Zbierz wszystkie punkty typu 1
points_3d = []

for marker_type in [1]:
    for folder_path in ['rat2\\Szkiełko 1', 'rat2\\Szkiełko 2']:
        ref_dict = load_ref_dict_from_txt(folder_path)
        for filename in os.listdir(folder_path):
            if filename.endswith('.xml'):
                for ref_key in ref_dict:
                    if filename.endswith(f"{ref_key}.xml"):
                        file_path = os.path.join(folder_path, filename)
                        tree = ET.parse(file_path)
                        root = tree.getroot()
                        calibrated_cor_dict = calibrate_with_rotation(root, ref_dict[ref_key])

                        if marker_type in calibrated_cor_dict:
                            x, y, z = calibrated_cor_dict[marker_type]
                            for xi, yi, zi in zip(x, y, z):
                                points_3d.append([xi, yi, zi])

# Konwertuj do numpy array
points_3d = np.array(points_3d)


In [5]:
import alphashape
import trimesh

# Wygeneruj alpha shape (dobierz alpha metodą prób, np. 1.0, 0.5, 0.1)
alpha = 0.5
shape = alphashape.alphashape(points_3d, alpha)

# Konwertuj do siatki i dodaj do plotly
if shape and not shape.is_empty:
    # Konwertuj do mesh3d przez Trimesh
    mesh = trimesh.Trimesh(vertices=np.array(shape.vertices), faces=np.array(shape.faces))
    fig.add_trace(go.Mesh3d(
        x=mesh.vertices[:, 0],
        y=mesh.vertices[:, 1],
        z=mesh.vertices[:, 2],
        i=mesh.faces[:, 0],
        j=mesh.faces[:, 1],
        k=mesh.faces[:, 2],
        color='lightgreen',
        opacity=0.4,
        name='Alpha Shape'
    ))
fig.show()